# Flint 2012 DREAM Data Preprocessing

In [1]:
import scipy.io as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.decomposition import PCA
from scipy.stats import zscore

### 1) Load data all at once to make loop run faster

In [2]:
files = np.empty(5, dtype=object)

for h in range(5):
    files[h] = sc.loadmat(f'Flint_2012_e{h+1}.mat')

### 2) Gather all data into two arrays

#### Helper vstack function

In [3]:
def vstack(top, bottom):
    if len(top) == 0:
        return bottom # vstack doesn't work with empty array
    else:
        return np.vstack((top, bottom))

In [4]:
'''
The data-wrangling code here is a bit messy, but it makes more sense when viewed as the
Python equivalent of the work done by the original authors.
'''

dataset_velocities = []
dataset_spikes = []

for file in files:
    
    nSubject = len(file['Subject'])


    # For each subject:
    for i in range(nSubject):
        print(f'Wrangling subject {i}... ')

        # Get all data pertaining to the i-th subject in this experiment.
        # Extra [0] added because Python yields file['Subject'] as a 2D array.
        Subject_i = file['Subject'][i][0]

        nTrial = len(Subject_i['Trial'])

        # Get the number of neurons from the first trial.
        # Extra [0] added at the end because Python wrapped the ['Neuron'] data in a 1-element array.
        nNeuron = len(Subject_i['Trial'][0]['Neuron'][0])

        
        
        vel = np.array([])
        spk = np.array([])

        # Agglomerate over trials
        for j in range(nTrial):

            Trial_j = Subject_i['Trial'][j]

            # Get [SOMETHING]
            # Extra [0] added because Python wrapped the ['Time'] data in a 1-element array.
            beats = len(Trial_j['Time'][0])

            # vel and Trial_j['HandVel'][0] are ?x3 arrays.
            # ? is related to how many timestamps there are, and 3 corresponds to the 3D velocity.
            # Extra [0] added because Python wrapped the ['HandVel'] data in a 1-element array.    
            vel = vstack(vel, Trial_j['HandVel'][0])


            spk_k = np.zeros((beats, nNeuron))

            for k in range(nNeuron):

                # Spike binning from the k-th neuron.

                # The two extra [0] were added because Python wrapped that ['Neuron'] and ['Spike'] data in 1-element arrays.
                spiketimes = Trial_j['Neuron'][0][k]['Spike'][0]

                # If there are no spiketimes, skip the k-th neuron.
                if len(spiketimes) > 0:
                    # The [:,0] was added because Python wrapped the 1-D spiketime data as a 2D column vector.
                    # The [0] at the end is to only keep bin counts, not bin intervals.
                    spk_k[:,k] = np.histogram(spiketimes[:,0], bins=beats)[0]

            spk = vstack(spk, spk_k)


        # Only keep the first two dimensions of 3D velocity, since the other was always constant (since the motion was 2D).
        vel = vel[:, :2]


        # Save the data from the i-th subject and h-th Flint file to its own unique spot in the dataset_? arrays.
        dataset_velocities.append(vel)
        dataset_spikes.append(spk)

Wrangling subject 0... 
Wrangling subject 0... 
Wrangling subject 1... 
Wrangling subject 0... 
Wrangling subject 1... 
Wrangling subject 2... 
Wrangling subject 3... 
Wrangling subject 0... 
Wrangling subject 1... 
Wrangling subject 2... 
Wrangling subject 0... 
Wrangling subject 1... 


### 3) Preprocess (filter) the data

In [5]:
dataset_velocities = np.array(dataset_velocities, dtype=object)
dataset_spikes = np.array(dataset_spikes, dtype=object)

In [6]:
nSets = len(dataset_velocities)

# Make arrays for preprocessed velocities + spikes
procd_velocities = []
procd_spikes = []


for i in range(nSets):
    vel = dataset_velocities[i]
    spk = dataset_spikes[i]
    
    # Downsample data to 100ms bins
    idx = np.arange(9, len(vel), 10) # step = 10 to ensure 100ms binlength.
    
    moving_sum = spk.cumsum(axis=0) # Calculate vertical cumsum
    moving_sum[10:] = moving_sum[10:] - moving_sum[:-10] # Use cumsum to get moving sum with window length 10.
    spk_ds = moving_sum[idx]
    
    vel_ds = vel[idx - 5] # Get the velocity reading halfway between each entry of spk_ds.
    
    # Hit neural data with PCA
    pca = PCA(n_components=10)
    spk_pca = pca.fit_transform(spk_ds) # spk_pca is the dimensionalty-reduced array of neural activity.
    
    # z-score the neural data
    spk_z = zscore(spk_pca, ddof=1)
    
    
    # Store the data
    procd_velocities.append(vel_ds)
    procd_spikes.append(spk_z)

### 4) Store the data in a file

In [7]:
procd_velocities = np.array(procd_velocities, dtype=object)
procd_spikes = np.array(procd_spikes, dtype=object)

In [8]:
np.save('procd_velocities', procd_velocities)
np.save('procd_spikes', procd_spikes)